# Cramer GAN implementation by tensorflow

In [ ]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# utility func

In [ ]:
# save metrics
def save_metrics(metrics, epoch=None):
    # make directory if there is not
    path = "metrics_celeb"
    if not os.path.isdir(path):
        os.makedirs(path)

    # save metrics
    plt.figure(figsize=(10,8))
    plt.plot(metrics["d_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "dloss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["g_loss"], label="generative loss", color="r")
    plt.legend()
    plt.savefig(os.path.join(path, "g_loss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["g_loss"], label="generative loss", color="r")
    plt.plot(metrics["d_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "both_loss" + str(epoch) + ".png"))
    plt.close()

In [ ]:
# plot images
def save_imgs(images, plot_dim=(8,2), size=(8,32), epoch=None):
    # make directory if there is not
    path = "generated_figures_celeb"
    if not os.path.isdir(path):
        os.makedirs(path)

    num_examples = plot_dim[0]*plot_dim[1]
    num_examples = 16
    fig = plt.figure(figsize=size)

    for i in range(num_examples):
        plt.subplot(plot_dim[0], plot_dim[1], i+1)
        img = images[i, :]
        img = img.reshape((1024, 1024, 3))
        plt.tight_layout()
        plt.imshow(img)
        plt.axis("off")
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(os.path.join(path, str(epoch) + ".png"))
    plt.close()

In [ ]:
# training
import pickle
import numpy as np
import os

def unpickle(file):
    fo = open(file, 'rb')
    #print(file)
    dict = pickle.load(fo, encoding='latin1')
    fo.close()
    return dict

def one_hot_vec(label):
    vec = np.zeros(10)
    vec[label] = 1
    return vec

def load_data():
    x_all = []
    y_all = []
    for i in range (5):
        d = unpickle("cifar-10-batches-py/data_batch_" + str(i+1))
        x_ = d['data']
        y_ = d['labels']
        x_all.append(x_)
        y_all.append(y_)

    d = unpickle('cifar-10-batches-py/test_batch')
    x_all.append(d['data'])
    y_all.append(d['labels'])

    x = np.concatenate(x_all) / np.float32(255)
    y = np.concatenate(y_all)
    x = np.dstack((x[:, :1024], x[:, 1024:2048], x[:, 2048:]))
    x = x.reshape((x.shape[0], 32, 32, 3))

    #pixel_mean = np.mean(x[0:50000],axis=0)
    #x -= pixel_mean
    y = np.array(list(map(one_hot_vec, y)))
    #X_train = x[0:50000,:,:,:]
    #Y_train = y[0:50000]
    #X_test = x[50000:,:,:,:]
    #Y_test = y[50000:]

    #return (X_train, Y_train, X_test, Y_test)
    return x, y

# model

In [ ]:
class Generator:
    def __init__(self):
        self.reuse = False
        self.initializer = tf.contrib.layers.xavier_initializer()
        self.X_dim = 1024*1024*3
        self.z_dim = 100

        self.depths = [1024, 512, 256, 128, 64, 32, 16, 8, 3]
        self.s_size = 6

    def __call__(self, z, training=False):
        with tf.variable_scope('g', reuse=self.reuse):
            inputs = tf.reshape(z, [-1, self.z_dim])
            fc1 = tf.layers.dense(inputs, 2*2*self.depths[1])
            fc1 = tf.reshape(fc1, [-1, 2,2,self.depths[1]])
            #fc1 = tf.layers.batch_normalization(fc1)
            fc1 = tf.nn.relu(fc1)
            conv1 = tf.contrib.layers.conv2d_transpose(fc1, self.depths[0], [4,4],[2,2]) # 4x4x1024
            conv1 = tf.nn.relu(conv1)
            conv2 = tf.contrib.layers.conv2d_transpose(conv1, self.depths[1], [4,4],[2,2]) # 8x8x512
            conv2 = tf.nn.relu(conv2)
            conv3 = tf.contrib.layers.conv2d_transpose(conv2, self.depths[2], [4,4],[2,2]) # 16x16x256
            conv3 = tf.nn.relu(conv3)
            conv4 = tf.contrib.layers.conv2d_transpose(conv3, self.depths[3], [4,4],[2,2]) # 32x32x128
            conv4 = tf.nn.relu(conv4)
            conv5 = tf.contrib.layers.conv2d_transpose(conv4, self.depths[4], [4,4],[2,2]) # 64x64x64
            conv5 = tf.nn.relu(conv5)
            conv6 = tf.contrib.layers.conv2d_transpose(conv5, self.depths[5], [4,4],[2,2]) # 128x128x32
            conv6 = tf.nn.relu(conv6)
            conv7 = tf.contrib.layers.conv2d_transpose(conv6, self.depths[6], [4,4],[2,2]) # 256x256x16
            conv7 = tf.nn.relu(conv7)
            conv8 = tf.contrib.layers.conv2d_transpose(conv7, self.depths[7], [4,4],[2,2]) # 512x512x8
            conv8 = tf.nn.relu(conv8)
            conv9 = tf.contrib.layers.conv2d_transpose(conv8, self.depths[8], [4,4],[2,2], activation_fn=tf.sigmoid) # 1024
            conv9 = tf.reshape(conv9, [-1, 1024*1024*3])
            outputs = conv9

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return outputs

In [ ]:
class Discriminator:
    def __init__(self):
        self.reuse = False
        self.X_dim = 1024*1024*3
        self.initializer = tf.contrib.layers.xavier_initializer()

        self.depths = [3, 8, 16, 32, 64, 128, 256, 512, 1024]
        self.s_size = 6

    def __call__(self, x, training=False, name=''):
        def leaky_relu(x, leak=0.2, name='outputs'):
            return tf.maximum(x, x * leak, name=name)

        with tf.name_scope('d' + name), tf.variable_scope('d', reuse=self.reuse):
            x = tf.reshape(x, [-1, 1024, 1024, 3])
            conv1 = tf.layers.conv2d(x, self.depths[1], [4,4], [2,2]) # 512x512x8
            conv1 = leaky_relu(conv1)
            conv2 = tf.layers.conv2d(conv1, self.depths[2], [4,4], [2,2]) # 256x256x16
            conv2 = leaky_relu(conv2)
            conv3 = tf.layers.conv2d(conv2, self.depths[3], [4,4], [2,2]) # 128x128x32
            conv3 = leaky_relu(conv3)
            conv4 = tf.layers.conv2d(conv3, self.depths[4], [4,4], [2,2]) # 64x64x64
            conv4 = leaky_relu(conv4)
            conv5 = tf.layers.conv2d(conv4, self.depths[5], [4,4], [2,2]) # 32x32x128
            conv5 = leaky_relu(conv5)
            conv6 = tf.layers.conv2d(conv5, self.depths[6], [4,4], [2,2]) # 16x16x256
            conv6 = leaky_relu(conv6)
            conv7 = tf.layers.conv2d(conv6, self.depths[7], [4,4], [2,2]) # 8x8x512
            conv7 = leaky_relu(conv7)
            conv8 = tf.layers.conv2d(conv7, self.depths[8], [4,4], [2,2]) # 4x4x1024
            conv8 = leaky_relu(conv8)
            conv8 = tf.contrib.layers.flatten(conv8) 
            #fc1 = tf.layers.dense(conv4, 512)
            #fc1 = leaky_relu(fc1)
            fc2 = tf.layers.dense(conv8, 256)
        outputs = fc2
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')
        return outputs

In [ ]:
# f(x) = ||h(x) - h(x_)|| - ||h(x)||
class Critic(object):
    def __init__(self, h):
        self.h = h
    def __call__(self, x, x_):
        return tf.norm(self.h(x) - self.h(x_), axis=1) - tf.norm(self.h(x), axis=1)

# f(x) = ||h(x) - h(x_)||
class calc_Norm(object):
    def __init__(self, h):
        self.h = h
    def __call__(self, x, x_):
        return tf.norm(self.h(x) -self.h(x_))

In [ ]:
class GAN:
    def __init__(self):

        self.batch_size = 32
        print("batch_size:" + str(self.batch_size))
        img_size = 1024
        self.z_dim = 100

        self.epochs = 1000000
        self.epoch_saveMetrics = 100
        self.epoch_saveSampleImg = 100
        self.epoch_saveParamter = 10000
        self.losses = {"d_loss":[], "g_loss":[]}

        self.x, self.y = load_data()
        self.X_tr = tf.placeholder(tf.float32, shape=[None, img_size, img_size, 3])
        self.Z1 = tf.placeholder(tf.float32, [None, self.z_dim])
        
        self.g = Generator()
        self.d = Discriminator()
        self.Xg = self.g(self.Z1)

    def loss(self):
        Xg = tf.reshape(self.Xg, [self.batch_size, -1]) 
        Xr = tf.reshape(self.X_tr, [self.batch_size, -1])

        epsilon = tf.random_uniform([], 0.0, 1.0) # 怪しい
        Xhat = epsilon * Xr + (1-epsilon) * Xg   
        dhat = self.d(Xhat)

        ddx = tf.gradients(dhat, Xhat)[0]
        #print((ddx.get_shape().as_list()))
        ddx = tf.norm(ddx, axis=1)
        ddx = tf.reduce_mean(tf.square(ddx - 1.0) * 10)      
  
        L_d = tf.reduce_mean(self.d(Xg)) - tf.reduce_mean(self.d(Xr)) + ddx
        L_g = -tf.reduce_mean(self.d(Xg))
        return L_g, L_d

    def train(self):
        # Optimizer
        learning_rate = 1e-4
        beta1 = 0.5
        beta2 = 0.9

        self.L_g, self.L_d = self.loss()

        d_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1, beta2=beta2)
        d_train_op = d_opt.minimize(self.L_d, var_list=self.d.variables)
        g_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1, beta2=beta2)
        g_train_op = g_opt.minimize(self.L_g, var_list=self.g.variables)

        saver = tf.train.Saver()
        #%debug
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            #rom tensorflow.python import debug as tf_debug
            #ess = tf_debug.LocalCLIDebugWrapperSession(sess)

            z_test = np.random.uniform(-1, 1, size=[
self.batch_size, self.z_dim])

            for epoch in range(self.epochs):

                # X_mb, Y_mbを収集
                def extractXimg(path, batch_size):
                    imgs = os.listdir(path)
                    rand_id = np.random.randint(0, len(imgs), size=batch_size)
                    X_mb = np.zeros((1, 1024, 1024, 3))

                    for i in range(batch_size):
                        img = Image.open(path+"/"+imgs[rand_id[i]])
                        
                        width, height = img.size
                        square_size = min(img.size)

                        if width > height:
                            top = 0
                            bottom = square_size
                            left = (width - square_size) / 2
                            right = left + square_size
                            box = (left, top, right, bottom)
                        else:
                            left = 0
                            right = square_size
                            top = (height - square_size) / 2
                            bottom = top + square_size
                            box = (left, top, right, bottom)

                        img = img.crop(box)
                        thumbnail_size = (1024, 1024)
                        img = img.resize(thumbnail_size, Image.ANTIALIAS)
                        #img.thumbnail(thumbnail_size)
                        #print(img.size)
                        img_np = np.asarray(img)
                        X_mb = np.vstack((X_mb, img_np[np.newaxis, :]))
                        img.close()
                    X_mb = X_mb[1:,:]
                    return X_mb
                
                for _ in range(5):
                    # 訓練データを抜粋
                    z1 = np.random.uniform(-1, 1, size=[self.batch_size, self.z_dim])
                    X_mb = extractXimg("./img_align_celeba", self.batch_size)
                    _, d_loss_value = sess.run([d_train_op, self.L_d], feed_dict={self.X_tr: X_mb, self.Z1:z1})

                # train G
                z1 = np.random.uniform(-1, 1, size=[self.batch_size, self.z_dim])
                X_mb = extractXimg("./img_align_celeba", self.batch_size)
                _, g_loss_value = sess.run([g_train_op, self.L_g], feed_dict={self.X_tr: X_mb, self.Z1: z1})

                # generate Sample Imgs
                #sampleImgsOfX2Y, sampleImgsOfY2X = sess.run([self.X2Y, self.Y2X], feed_dict={self.X_tr: X_mb, self.Y_tr: Y_mb})

                # 結果をappend
                self.losses["d_loss"].append(np.sum(d_loss_value))
                self.losses["g_loss"].append(np.sum(g_loss_value))
                
                if epoch % 100 == 0:
                    print("epoch:" + str(epoch))

                # lossの可視化
                if epoch % self.epoch_saveMetrics == 1:
                    save_metrics(self.losses, epoch)

                # 画像の変換テスト
                if epoch % self.epoch_saveSampleImg == 0:
                    img = sess.run(self.Xg, feed_dict={self.Z1: z_test})
                    save_imgs(img, epoch=str(epoch))

                # parameterのsave
                if epoch % self.epoch_saveParamter == 1:
                    path = "celeb"
                    if not os.path.isdir(path):
                        os.makedirs(path)

                    saver.save(sess, "./" + path + "/WGANgp_model" + str(epoch) + ".ckpt")
       

    def sample_images(self, row=5, col=12, inputs=None, epoch=None):
        images = self.g(inputs, training=True)
        return images

In [ ]:
gan = GAN()
gan.train()